# Import Libs

In [1]:
import csv
import pandas as pd # for data manipulation 
import numpy as np
import os, sys, glob, math, pickle
from tqdm import tqdm

# This function helps to calculate probability distribution, which goes into BBN (note, can handle up to 2 parents)
def cpt_probs(df, child, parents):
    try:
        # dependencies_arr = [pd.Categorical(df[parent],categories=df[parent].cat.categories.tolist()) for parent in parents]
        dependencies_arr = [df[parent] for parent in parents]
        # cpt = pd.crosstab(dependencies_arr, df[child], rownames=parents, colnames=[child], margins=False, normalize='index', dropna=False).sort_index().to_numpy().reshape(-1).tolist()
        cpt = pd.crosstab(dependencies_arr, df[child], rownames=parents, colnames=[child], margins=False, normalize='index', dropna=False).sort_index()
        return cpt
    except Exception as err:
        print(err)
        return None 

def cpt_probs_freq(df, child, parents):
    try:
        # dependencies_arr = [pd.Categorical(df[parent],categories=df[parent].cat.categories.tolist()) for parent in parents]
        dependencies_arr = [df[parent] for parent in parents]
        # cpt = pd.crosstab(dependencies_arr, df[child], rownames=parents, colnames=[child], margins=False, normalize='index', dropna=False).sort_index().to_numpy().reshape(-1).tolist()
        cpt = pd.crosstab(dependencies_arr, df[child], rownames=parents, colnames=[child], margins=False, dropna=False).sort_index()
        return cpt
    except Exception as err:
        print(err)
        return None 

def euclidean_dist(row):
    # Function to calc euclidean distance on every df row 
    euc_dist = math.sqrt(row["U2G_Distance"]**2 - row["Height"]**2)
    return euc_dist

# Main Training Steps

## Compile the processed data

In [2]:
dl_df_8uav = pd.read_hdf("/media/research-student/One Touch/FANET Datasets/Dataset_NP10000_BPSK_6-5Mbps/Dataset_NP10000_BPSK_6-5Mbps_8UAVs_processed_downlink.h5", '8_UAVs')
dl_df_16uav = pd.read_hdf("/media/research-student/One Touch/FANET Datasets/Dataset_NP10000_BPSK_6-5Mbps/Dataset_NP10000_BPSK_6-5Mbps_16UAVs_processed_downlink.h5", '16_UAVs')
dl_df_24uav = pd.read_hdf("/media/research-student/One Touch/FANET Datasets/Dataset_NP10000_BPSK_6-5Mbps/Dataset_NP10000_BPSK_6-5Mbps_24UAVs_processed_downlink.h5", '24_UAVs')
dl_df_32uav = pd.read_hdf("/media/research-student/One Touch/FANET Datasets/Dataset_NP10000_BPSK_6-5Mbps/Dataset_NP10000_BPSK_6-5Mbps_32UAVs_processed_downlink.h5", '32_UAVs')
dl_df_40uav = pd.read_hdf("/media/research-student/One Touch/FANET Datasets/Dataset_NP10000_BPSK_6-5Mbps/Dataset_NP10000_BPSK_6-5Mbps_40UAVs_processed_downlink.h5", '40_UAVs')
dl_df = pd.concat([dl_df_8uav, dl_df_16uav, dl_df_24uav, dl_df_32uav, dl_df_40uav], ignore_index=True)

## Discretize the input layer. Change the df below to either DOWNLINK / UPLINK DF.

In [3]:
delay_threshold = 0.04
df = dl_df # NOTE: Change this to either dl_df / ul_df

df = df[df['U2G_SINR'].notna()] # Filter out rows with missing crucial information
classes_df = pd.DataFrame() # Created an empty df to store classes data to reduce size of df that need to work with
# First, discretise the values to classes
h_dist_range = 500
h_dist_num_classes = 100
h_dist_labels = [str(num) for num in np.arange(0,h_dist_num_classes)+1]
# h_dist_labels = ['vs','s','m','l','vl']
height_labels = ['vs','s','m','l','vl']
num_members_labels = ['vs','s','m','l','vl']
sending_interval_labels = ['vs','s','m','l','vl']
pkt_size_labels = ['vs','s','m','l','vl']
sinr_labels = ['vs','s','m','l','vl']
delay_labels = ['vs','s','m','l','vl']
throughput_labels = ['s','m','l']
queueing_labels = ['s','m','l']
ber_labels = ['vs','s','m','l','vl']
jitter_labels = ['s','m','l']
# Independent vars
h_dist_class_bnd = np.linspace(0, h_dist_range, h_dist_num_classes, endpoint=False).tolist()
h_dist_class_bnd.append(h_dist_range+1)
classes_df["H_Dist_Class"] = pd.cut(df.U2G_H_Dist, h_dist_class_bnd, right=False, include_lowest=True, labels=h_dist_labels)
# classes_df["H_Dist_Class"] = pd.cut(df.U2G_H_Dist, [0,100,200,300,400,501], right=False, include_lowest=True, labels=h_dist_labels)
classes_df["Height_Class"] = pd.cut(df.Height, [1,25,49,73,97,121], right=False, include_lowest=True, labels=height_labels)
classes_df["Num_Members_Class"] = pd.cut(df.Num_Members, [2,8,16,24,32,40], right=False, include_lowest=True, labels=num_members_labels)
# classes_df["Sending_Interval_Class"] = pd.cut(df.Mean_Sending_Interval, [40,232,424,616,808,1001], right=True, include_lowest=True, labels=sending_interval_labels)
# classes_df["Packet_Size_Class"] = pd.cut(df.Bytes, [24,248,472,696,920,1145], right=True, include_lowest=True, labels=pkt_size_labels)
classes_df["Sending_Interval_Class"] = pd.cut(df.Mean_Sending_Interval, [40,232,424,616,808,1001], right=False, include_lowest=True, labels=sending_interval_labels)
classes_df["Packet_Size_Class"] = pd.cut(df.Bytes, [24,248,472,696,920,1145], right=False, include_lowest=True, labels=pkt_size_labels)
# Second layer
classes_df["SINR_Class"], sinr_bins = pd.qcut(df.U2G_SINR, q=5, labels=sinr_labels, retbins=True)
classes_df["Delay_Class"], delay_bins = pd.qcut(df.Delay, q=5, labels=delay_labels, retbins=True)
classes_df["Throughput_Class"], throughput_bins = pd.qcut(df.Throughput, q=3, labels=throughput_labels, duplicates='drop', retbins=True)
classes_df["Queueing_Time_Class"], queueing_time_bins = pd.qcut(df.Queueing_Time, q=3, labels=queueing_labels, duplicates='drop', retbins=True)
classes_df["BER_Class"], ber_bins = pd.qcut(df.U2G_BER, q=5, labels=ber_labels, retbins=True)
classes_df["Jitter_Class"], jitter_bins = pd.qcut(df.Jitter, q=3, labels=jitter_labels, retbins=True)
classes_df["Reliable"] = (df["Packet_State"] == "Reliable")
classes_df["Delay_Exceeded"] = (df["Delay"] >= delay_threshold)
classes_df["Retry_Count"] = df["Retry_Count"] #NOTE: Retry_Count is not necessarily the same as number of dropped packets
classes_df["Incorrectly_Received"] = df["Incorrectly_Received"]
classes_df["Queue_Overflow"] = df["Queue_Overflow"]

In [5]:
# Save classes_df to csv for examination
data_type = "Downlink"
classes_df.to_csv("/media/research-student/One Touch/FANET Datasets/Dataset_NP10000_BPSK_6-5Mbps/classes_df_{}_hdist_{}_classes.h5".format(data_type, h_dist_num_classes))

In [30]:
# Save classes_df to hdf format for convenient loading
data_type = "Downlink"
classes_df.to_hdf("/media/research-student/One Touch/FANET Datasets/Dataset_NP10000_BPSK_6-5Mbps/classes_df_{}_hdist_{}_classes.h5".format(data_type, h_dist_num_classes), key='Downlink', format='table')

In [31]:
# Save the qcut bin intervals for later use
np.save("/home/research-student/omnet-fanet/cpt/{}_Basic_hdist_{}_classes/sinr_bins_dl.npy".format(data_type, h_dist_num_classes), sinr_bins)
np.save("/home/research-student/omnet-fanet/cpt/{}_Basic_hdist_{}_classes/delay_bins_dl.npy".format(data_type, h_dist_num_classes), delay_bins)
np.save("/home/research-student/omnet-fanet/cpt/{}_Basic_hdist_{}_classes/throughput_bins_dl.npy".format(data_type, h_dist_num_classes), throughput_bins)
np.save("/home/research-student/omnet-fanet/cpt/{}_Basic_hdist_{}_classes/queueing_time_bins_dl.npy".format(data_type, h_dist_num_classes), queueing_time_bins)
np.save("/home/research-student/omnet-fanet/cpt/{}_Basic_hdist_{}_classes/ber_bins_dl.npy".format(data_type, h_dist_num_classes), ber_bins)
np.save("/home/research-student/omnet-fanet/cpt/{}_Basic_hdist_{}_classes/jitter_bins_dl.npy".format(data_type, h_dist_num_classes), jitter_bins)

In [3]:
# Load classes_df for later parts (if previous part not run)
classes_df = pd.read_hdf("/media/research-student/One Touch/FANET Datasets/Dataset_NP10000_BPSK_6-5Mbps/classes_df_downlink.h5", 'Downlink')

delay_threshold = 0.04
# First, discretise the values to classes
h_dist_labels = ['vs','s','m','l','vl']
height_labels = ['vs','s','m','l','vl']
num_members_labels = ['vs','s','m','l','vl']
# num_members_labels = ['vs','s','m','l']
sending_interval_labels = ['vs','s','m','l','vl']
pkt_size_labels = ['vs','s','m','l','vl']
sinr_labels = ['vs','s','m','l','vl']
delay_labels = ['vs','s','m','l','vl']
throughput_labels = ['s','m','l']
queueing_labels = ['s','m','l']
ber_labels = ['vs','s','m','l','vl']
jitter_labels = ['s','m','l']

## Train Fully Connected BN

The following computes the CPT of each node using the "Unknown" class

In [ ]:
# Calculate the conditional probabilities table for each second layer class
parents_1 = ["H_Dist_Class", "Height_Class", "Num_Members_Class", "Sending_Interval_Class", "Packet_Size_Class"] 
sinr_cpt = cpt_probs(classes_df, child="SINR_Class", parents=parents_1)
delay_cpt = cpt_probs(classes_df, child="Delay_Class", parents=parents_1)
throughput_cpt = cpt_probs(classes_df, child="Throughput_Class", parents=parents_1)
queueing_cpt = cpt_probs(classes_df, child="Queueing_Time_Class", parents=parents_1)
ber_cpt = cpt_probs(classes_df, child="BER_Class", parents=parents_1)
jitter_cpt = cpt_probs(classes_df, child="Jitter_Class", parents=parents_1)

# parents_1_labels = [h_dist_labels,height_labels,num_members_labels,sending_interval_labels,pkt_size_labels]
parents_1_labels = [h_dist_labels,height_labels,num_members_labels,sending_interval_labels,pkt_size_labels]
sinr_cpt = sinr_cpt.reindex(pd.MultiIndex.from_product(parents_1_labels, names=parents_1))[sinr_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network
delay_cpt = delay_cpt.reindex(pd.MultiIndex.from_product(parents_1_labels, names=parents_1))[delay_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network
throughput_cpt = throughput_cpt.reindex(pd.MultiIndex.from_product(parents_1_labels, names=parents_1))[throughput_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network
queueing_cpt = queueing_cpt.reindex(pd.MultiIndex.from_product(parents_1_labels, names=parents_1))[queueing_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network
ber_cpt = ber_cpt.reindex(pd.MultiIndex.from_product(parents_1_labels, names=parents_1))[ber_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network
jitter_cpt = jitter_cpt.reindex(pd.MultiIndex.from_product(parents_1_labels, names=parents_1))[jitter_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network

parents_2 = ["SINR_Class", "BER_Class", "Delay_Class", "Throughput_Class", "Queueing_Time_Class", "Jitter_Class"]
incorrect_rcvd_cpt = cpt_probs(classes_df, child="Incorrectly_Received", parents=parents_2)
incorrect_rcvd_cpt["Unknown"] = 0
incorrect_rcvd_cpt.loc[(incorrect_rcvd_cpt==0).all(axis=1).values, "Unknown"] = 1
delay_exceeded_cpt = cpt_probs(classes_df, child="Delay_Exceeded", parents=parents_2)
delay_exceeded_cpt.rename(columns = {True:'True'}, inplace = True)
delay_exceeded_cpt.rename(columns = {False:'False'}, inplace = True)
delay_exceeded_cpt["Unknown"] = 0
delay_exceeded_cpt.loc[(delay_exceeded_cpt==0).all(axis=1).values, "Unknown"] = 1
queue_overflow_cpt = cpt_probs(classes_df, child="Queue_Overflow", parents=parents_2)
queue_overflow_cpt["Unknown"] = 0
queue_overflow_cpt.loc[(queue_overflow_cpt==0).all(axis=1).values, "Unknown"] = 1
# TODO: TEMPORARY
rel_cpt = cpt_probs(classes_df, child="Reliable", parents=parents_2)
rel_cpt["Unknown"] = 0
rel_cpt.loc[(rel_cpt==0).all(axis=1).values, "Unknown"] = 1
# Add unknown "U" class for cases that are not seen in data
parents_2_labels = [sinr_labels,delay_labels,throughput_labels,queueing_labels,ber_labels,jitter_labels]
incorrect_rcvd_labels = incorrect_rcvd_cpt.columns.values
delay_exceeded_labels = delay_exceeded_cpt.columns.values
queue_overflow_labels = queue_overflow_cpt.columns.values
incorrect_rcvd_cpt = incorrect_rcvd_cpt.reindex(pd.MultiIndex.from_product(parents_2_labels, names=parents_2))[incorrect_rcvd_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network
delay_exceeded_cpt = delay_exceeded_cpt.reindex(pd.MultiIndex.from_product(parents_2_labels, names=parents_2))[delay_exceeded_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network
queue_overflow_cpt = queue_overflow_cpt.reindex(pd.MultiIndex.from_product(parents_2_labels, names=parents_2))[queue_overflow_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network
# TODO: TEMPORARY
rel_labels = rel_cpt.columns.values
rel_cpt = rel_cpt.reindex(pd.MultiIndex.from_product(parents_2_labels, names=parents_2))[rel_labels]

parents_3 = ["Incorrectly_Received", "Delay_Exceeded", "Queue_Overflow"]
reliability_cpt = cpt_probs(classes_df, child="Reliable", parents=parents_3)
reliability_cpt.rename(columns = {True:'True'}, inplace = True)
reliability_cpt.rename(columns = {False:'False'}, inplace = True)
reliability_cpt["Unknown"] = 0
reliability_cpt.loc[(reliability_cpt==0).all(axis=1).values, "Unknown"] = 1

# TODO: Fix the below
# parents_3_labels = [incorrect_rcvd_labels,delay_exceeded_labels,queue_overflow_labels]
# reliability_labels = reliability_cpt.columns.values
# reliability_cpt = reliability_cpt.reindex(pd.MultiIndex.from_product(parents_3_labels, names=parents_3))[reliability_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network

The following computes the CPT WITHOUT using the "Unknown" class, but using pandas.fillna instead.

In [6]:
# Calculate the conditional probabilities table for each second layer class
parents_1 = ["H_Dist_Class", "Height_Class", "Num_Members_Class", "Sending_Interval_Class", "Packet_Size_Class"] 
sinr_cpt = cpt_probs(classes_df, child="SINR_Class", parents=parents_1)
delay_cpt = cpt_probs(classes_df, child="Delay_Class", parents=parents_1)
throughput_cpt = cpt_probs(classes_df, child="Throughput_Class", parents=parents_1)
queueing_cpt = cpt_probs(classes_df, child="Queueing_Time_Class", parents=parents_1)
ber_cpt = cpt_probs(classes_df, child="BER_Class", parents=parents_1)
jitter_cpt = cpt_probs(classes_df, child="Jitter_Class", parents=parents_1)

# parents_1_labels = [h_dist_labels,height_labels,num_members_labels,sending_interval_labels,pkt_size_labels]
parents_1_labels = [h_dist_labels,height_labels,num_members_labels,sending_interval_labels,pkt_size_labels]
sinr_cpt = sinr_cpt.reindex(pd.MultiIndex.from_product(parents_1_labels, names=parents_1))[sinr_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network
delay_cpt = delay_cpt.reindex(pd.MultiIndex.from_product(parents_1_labels, names=parents_1))[delay_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network
throughput_cpt = throughput_cpt.reindex(pd.MultiIndex.from_product(parents_1_labels, names=parents_1))[throughput_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network
queueing_cpt = queueing_cpt.reindex(pd.MultiIndex.from_product(parents_1_labels, names=parents_1))[queueing_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network
ber_cpt = ber_cpt.reindex(pd.MultiIndex.from_product(parents_1_labels, names=parents_1))[ber_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network
jitter_cpt = jitter_cpt.reindex(pd.MultiIndex.from_product(parents_1_labels, names=parents_1))[jitter_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network

parents_2 = ["SINR_Class", "BER_Class", "Delay_Class", "Throughput_Class", "Queueing_Time_Class", "Jitter_Class"]
incorrect_rcvd_cpt = cpt_probs(classes_df, child="Incorrectly_Received", parents=parents_2)
incorrect_rcvd_cpt.loc[(incorrect_rcvd_cpt==0).all(axis=1).values] = np.NaN
delay_exceeded_cpt = cpt_probs(classes_df, child="Delay_Exceeded", parents=parents_2)
delay_exceeded_cpt.rename(columns = {True:'True'}, inplace = True)
delay_exceeded_cpt.rename(columns = {False:'False'}, inplace = True)
delay_exceeded_cpt.loc[(delay_exceeded_cpt==0).all(axis=1).values] = np.NaN
queue_overflow_cpt = cpt_probs(classes_df, child="Queue_Overflow", parents=parents_2)
queue_overflow_cpt.loc[(queue_overflow_cpt==0).all(axis=1).values] = np.NaN
# TODO: TEMPORARY ------------------------------------------
# rel_cpt = cpt_probs(classes_df, child="Reliable", parents=parents_2)
# rel_cpt.loc[(rel_cpt==0).all(axis=1).values] = np.NaN
# ----------------------------------------------------------
parents_2_labels = [sinr_labels,delay_labels,throughput_labels,queueing_labels,ber_labels,jitter_labels]
incorrect_rcvd_labels = incorrect_rcvd_cpt.columns.values
delay_exceeded_labels = delay_exceeded_cpt.columns.values
queue_overflow_labels = queue_overflow_cpt.columns.values
incorrect_rcvd_cpt = incorrect_rcvd_cpt.reindex(pd.MultiIndex.from_product(parents_2_labels, names=parents_2))[incorrect_rcvd_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network
delay_exceeded_cpt = delay_exceeded_cpt.reindex(pd.MultiIndex.from_product(parents_2_labels, names=parents_2))[delay_exceeded_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network
queue_overflow_cpt = queue_overflow_cpt.reindex(pd.MultiIndex.from_product(parents_2_labels, names=parents_2))[queue_overflow_labels] # Rearrange rows and columns of cpt. This is important when building the Bayesian Network
# TODO: TEMPORARY
# rel_labels = rel_cpt.columns.values
# rel_cpt = rel_cpt.reindex(pd.MultiIndex.from_product(parents_2_labels, names=parents_2))[rel_labels]

# Use fillna to fill missing CPT rows -----------------------------------------------------------------
# NOTE: It is important that the CPTs are reindexed first, as this affects how they are filled
# Do forward fill first, then backward fill
incorrect_rcvd_cpt.fillna(method="ffill", inplace=True)
incorrect_rcvd_cpt.fillna(method="bfill", inplace=True)
delay_exceeded_cpt.fillna(method="ffill", inplace=True)
delay_exceeded_cpt.fillna(method="bfill", inplace=True)
queue_overflow_cpt.fillna(method="ffill", inplace=True)
queue_overflow_cpt.fillna(method="bfill", inplace=True)
# rel_cpt.fillna(method="ffill", inplace=True)
# rel_cpt.fillna(method="bfill", inplace=True)

# Do backward fill first, then forward fill
# incorrect_rcvd_cpt.fillna(method="bfill", inplace=True)
# incorrect_rcvd_cpt.fillna(method="ffill", inplace=True)
# delay_exceeded_cpt.fillna(method="bfill", inplace=True)
# delay_exceeded_cpt.fillna(method="ffill", inplace=True)
# queue_overflow_cpt.fillna(method="bfill", inplace=True)
# queue_overflow_cpt.fillna(method="ffill", inplace=True)
# rel_cpt.fillna(method="bfill", inplace=True)
# rel_cpt.fillna(method="ffill", inplace=True)
# ----------------------------------------------------------------------------------------------------



In [7]:
parents_3 = ["Incorrectly_Received", "Delay_Exceeded", "Queue_Overflow"]
reliability_cpt = cpt_probs(classes_df, child="Reliable", parents=parents_3)
reliability_cpt.rename(columns = {True:'True'}, inplace = True)
reliability_cpt.rename(columns = {False:'False'}, inplace = True)
reliability_cpt.loc[(reliability_cpt==0).all(axis=1).values] = np.NaN

# Use fillna to fill missing CPT rows -----------------------------------------------------------------
# NOTE: It is important that the CPTs are reindexed first, as this affects how they are filled
# Do forward fill first, then backward fill
reliability_cpt.fillna(method="ffill", inplace=True)
reliability_cpt.fillna(method="bfill", inplace=True)

# Do backward fill first, then forward fill
# reliability_cpt.fillna(method="bfill", inplace=True)
# reliability_cpt.fillna(method="ffill", inplace=True)
# ----------------------------------------------------------------------------------------------------

Save CPT Files as CSVs

In [8]:
data_type = "Downlink"
sinr_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}/sinr_cpt.csv".format(data_type))
delay_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}/delay_cpt.csv".format(data_type))
throughput_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}/throughput_cpt.csv".format(data_type))
queueing_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}/queueing_cpt.csv".format(data_type))
ber_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}/ber_cpt.csv".format(data_type))
jitter_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}/jitter_cpt.csv".format(data_type))

In [9]:
data_type = "Downlink"
incorrect_rcvd_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}/incorrect_rcvd_cpt.csv".format(data_type))
delay_exceeded_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}/delay_exceeded_cpt.csv".format(data_type))
queue_overflow_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}/queue_overflow_cpt.csv".format(data_type))

In [10]:
data_type = "Downlink"
reliability_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}/reliability_cpt.csv".format(data_type))

In [16]:
rel_cpt.to_csv("/home/research-student/omnet-fanet/cpt/rel_cpt.csv")

Save CPT Files as Pickle Files

In [17]:
data_type = "Downlink"
sinr_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}/sinr_cpt.pkl".format(data_type))
delay_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}/delay_cpt.pkl".format(data_type))
throughput_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}/throughput_cpt.pkl".format(data_type))
queueing_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}/queueing_cpt.pkl".format(data_type))
ber_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}/ber_cpt.pkl".format(data_type))
jitter_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}/jitter_cpt.pkl".format(data_type))

In [18]:
data_type = "Downlink"
incorrect_rcvd_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}/incorrect_rcvd_cpt.pkl".format(data_type))
delay_exceeded_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}/delay_exceeded_cpt.pkl".format(data_type))
queue_overflow_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}/queue_overflow_cpt.pkl".format(data_type))

In [19]:
data_type = "Downlink"
reliability_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}/reliability_cpt.pkl".format(data_type))

Extra Stuff

In [5]:
parents_1 = ["H_Dist_Class", "Height_Class", "Num_Members_Class", "Sending_Interval_Class", "Packet_Size_Class"] 
reliability_cpt_non_normal = cpt_probs_freq(classes_df, child="Reliable", parents=parents_1)
reliability_cpt_non_normal.to_csv("/home/research-student/omnet-fanet/cpt/reliability_freq.csv")
reliability_cpt_non_normal.to_pickle("/home/research-student/omnet-fanet/cpt/reliability_freq.pkl")

In [6]:
parents_1 = ["H_Dist_Class", "Height_Class", "Num_Members_Class", "Sending_Interval_Class", "Packet_Size_Class"] 
reliability_cpt_non_normal = cpt_probs(classes_df, child="Reliable", parents=parents_1)
reliability_cpt_non_normal.to_csv("/home/research-student/omnet-fanet/cpt/reliability_basic.csv")
reliability_cpt_non_normal.to_pickle("/home/research-student/omnet-fanet/cpt/reliability_basic.pkl")

## Training the 'basic' model

In [ ]:
# Load classes_df for later parts (if previous part not run)
data_type = "Downlink"
h_dist_num_classes = 25
classes_df = pd.read_hdf("/media/research-student/One Touch/FANET Datasets/Dataset_NP10000_BPSK_6-5Mbps/classes_df_{}_hdist_{}_classes.h5".format(data_type, h_dist_num_classes))

delay_threshold = 0.04
# First, discretise the values to classes
h_dist_labels = ['vs','s','m','l','vl']
height_labels = ['vs','s','m','l','vl']
num_members_labels = ['vs','s','m','l','vl']
sending_interval_labels = ['vs','s','m','l','vl']
pkt_size_labels = ['vs','s','m','l','vl']

In [4]:
parents_1 = ["H_Dist_Class", "Height_Class", "Num_Members_Class", "Sending_Interval_Class", "Packet_Size_Class"] 
incorrect_rcvd_cpt = cpt_probs(classes_df, child="Incorrectly_Received", parents=parents_1)
delay_exceeded_cpt = cpt_probs(classes_df, child="Delay_Exceeded", parents=parents_1)
delay_exceeded_cpt.rename(columns = {True:'True'}, inplace = True)
delay_exceeded_cpt.rename(columns = {False:'False'}, inplace = True)
queue_overflow_cpt = cpt_probs(classes_df, child="Queue_Overflow", parents=parents_1)

parents_3 = ["Incorrectly_Received", "Delay_Exceeded", "Queue_Overflow"]
reliability_cpt = cpt_probs(classes_df, child="Reliable", parents=parents_3)
reliability_cpt.rename(columns = {True:'True'}, inplace = True)
reliability_cpt.rename(columns = {False:'False'}, inplace = True)

In [5]:
# Save as CSV
data_type = "Downlink"
incorrect_rcvd_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}_Basic_hdist_{}_classes/incorrect_rcvd_cpt.csv".format(data_type, h_dist_num_classes))
delay_exceeded_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}_Basic_hdist_{}_classes/delay_exceeded_cpt.csv".format(data_type, h_dist_num_classes))
queue_overflow_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}_Basic_hdist_{}_classes/queue_overflow_cpt.csv".format(data_type, h_dist_num_classes))
reliability_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}_Basic_hdist_{}_classes/reliability_cpt.csv".format(data_type, h_dist_num_classes))

# Save as pickle
incorrect_rcvd_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}_Basic_hdist_{}_classes/incorrect_rcvd_cpt.pkl".format(data_type, h_dist_num_classes))
delay_exceeded_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}_Basic_hdist_{}_classes/delay_exceeded_cpt.pkl".format(data_type, h_dist_num_classes))
queue_overflow_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}_Basic_hdist_{}_classes/queue_overflow_cpt.pkl".format(data_type, h_dist_num_classes))
reliability_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}_Basic_hdist_{}_classes/reliability_cpt.pkl".format(data_type, h_dist_num_classes))

Trimmed BN Incorrect Rcvd

In [4]:
parents_trimmed = ["H_Dist_Class", "Height_Class"] 
incorrect_rcvd_trimmed_cpt = cpt_probs(classes_df, child="Incorrectly_Received", parents=parents_trimmed)
data_type = "Downlink"
incorrect_rcvd_trimmed_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}_Basic/incorrect_rcvd_trimmed_cpt.csv".format(data_type))
incorrect_rcvd_trimmed_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}_Basic/incorrect_rcvd_trimmed_cpt.pkl".format(data_type))

'Basic' BN model for single experiment (Single class for height, no. UAVs, sending interval and packet size)

In [7]:
dl_df = pd.read_hdf("/media/research-student/One Touch/FANET Datasets/Dataset_NP50000_BPSK_6-5Mbps_8UAVs_additional_data_processed_downlink.h5", '8_UAVs')
# OPTIONAL: Use only a subset of the data.------------------------------------------------
# # Let's split the dataset into 5 even sets
# df_set_1 = dl_df.iloc[::5, :].copy()
# df_set_2 = dl_df.iloc[1::5, :].copy()
# df_set_3 = dl_df.iloc[2::5, :].copy()
# df_set_4 = dl_df.iloc[3::5, :].copy()
# df_set_5 = dl_df.iloc[4::5, :].copy()
# dl_df = pd.concat([df_set_1, df_set_2, df_set_3, df_set_4]) # Use 80% of the data
# # dl_df = pd.concat([df_set_1, df_set_3, df_set_5]) # Use 60% of the data
# # dl_df = pd.concat([df_set_2, df_set_4]) # Use 60% of the data
# # dl_df = df_set_3 # Use 20% of the data
# ----------------------------------------------------------------------------------------

delay_threshold = 0.04
df = dl_df # NOTE: Change this to either dl_df / ul_df
df = df[df['U2G_SINR'].notna()] # Filter out rows with missing crucial information
classes_df = pd.DataFrame() # Created an empty df to store classes data to reduce size of df that need to work with
# First, discretise the values to classes
h_dist_range = 500
h_dist_num_classes = 200
h_dist_labels = [str(num) for num in np.arange(0,h_dist_num_classes)+1]
# Independent vars
h_dist_class_bnd = np.linspace(0, h_dist_range, h_dist_num_classes, endpoint=False).tolist()
h_dist_class_bnd.append(h_dist_range+1)
classes_df["H_Dist_Class"] = pd.cut(df.U2G_H_Dist, h_dist_class_bnd, right=False, include_lowest=True, labels=h_dist_labels)
classes_df["Reliable"] = (df["Packet_State"] == "Reliable")
classes_df["Delay_Exceeded"] = (df["Delay"] >= delay_threshold)
classes_df["Incorrectly_Received"] = df["Incorrectly_Received"]
classes_df["Queue_Overflow"] = df["Queue_Overflow"]

# Compute CPTs ---------------------------------------------------------------------------
parents_1 = ["H_Dist_Class"] 
incorrect_rcvd_cpt = cpt_probs(classes_df, child="Incorrectly_Received", parents=parents_1)
delay_exceeded_cpt = cpt_probs(classes_df, child="Delay_Exceeded", parents=parents_1)
delay_exceeded_cpt.rename(columns = {True:'True'}, inplace = True)
delay_exceeded_cpt.rename(columns = {False:'False'}, inplace = True)
queue_overflow_cpt = cpt_probs(classes_df, child="Queue_Overflow", parents=parents_1)

parents_3 = ["Incorrectly_Received", "Delay_Exceeded", "Queue_Overflow"]
reliability_cpt = cpt_probs(classes_df, child="Reliable", parents=parents_3)
reliability_cpt.rename(columns = {True:'True'}, inplace = True)
reliability_cpt.rename(columns = {False:'False'}, inplace = True)

# Save as CSV ----------------------------------------------------------------------------
data_type = "Downlink"
incorrect_rcvd_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}_Basic_Exp1_hdist_{}_classes/incorrect_rcvd_cpt.csv".format(data_type, h_dist_num_classes))
delay_exceeded_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}_Basic_Exp1_hdist_{}_classes/delay_exceeded_cpt.csv".format(data_type, h_dist_num_classes))
queue_overflow_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}_Basic_Exp1_hdist_{}_classes/queue_overflow_cpt.csv".format(data_type, h_dist_num_classes))
reliability_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}_Basic_Exp1_hdist_{}_classes/reliability_cpt.csv".format(data_type, h_dist_num_classes))

# Save as pickle
incorrect_rcvd_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}_Basic_Exp1_hdist_{}_classes/incorrect_rcvd_cpt.pkl".format(data_type, h_dist_num_classes))
delay_exceeded_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}_Basic_Exp1_hdist_{}_classes/delay_exceeded_cpt.pkl".format(data_type, h_dist_num_classes))
queue_overflow_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}_Basic_Exp1_hdist_{}_classes/queue_overflow_cpt.pkl".format(data_type, h_dist_num_classes))
reliability_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}_Basic_Exp1_hdist_{}_classes/reliability_cpt.pkl".format(data_type, h_dist_num_classes))

## Training v2 BN Model

In [2]:
# Load classes_df for later parts (if previous part not run)
classes_df = pd.read_hdf("/media/research-student/One Touch/FANET Datasets/Dataset_NP10000_BPSK_6-5Mbps/classes_df_downlink.h5", 'Downlink')

delay_threshold = 0.04
# First, discretise the values to classes
h_dist_labels = ['vs','s','m','l','vl']
height_labels = ['vs','s','m','l','vl']
num_members_labels = ['vs','s','m','l','vl']
sending_interval_labels = ['vs','s','m','l','vl']
pkt_size_labels = ['vs','s','m','l','vl']
sinr_labels = ['vs','s','m','l','vl']
delay_labels = ['vs','s','m','l','vl']
ber_labels = ['vs','s','m','l','vl']

In [3]:
sinr_cpt = cpt_probs(classes_df, child="SINR_Class", parents=["H_Dist_Class", "Height_Class"])
ber_cpt = cpt_probs(classes_df, child="BER_Class", parents=["SINR_Class"])
delay_cpt = cpt_probs(classes_df, child="Delay_Class", parents=["Num_Members_Class", "Sending_Interval_Class", "Packet_Size_Class", "BER_Class"])

incorrect_rcvd_cpt = cpt_probs(classes_df, child="Incorrectly_Received", parents=["BER_Class"])
delay_exceeded_cpt = cpt_probs(classes_df, child="Delay_Exceeded", parents=["Delay_Class"])
delay_exceeded_cpt.rename(columns = {True:'True'}, inplace = True)
delay_exceeded_cpt.rename(columns = {False:'False'}, inplace = True)
queue_overflow_cpt = cpt_probs(classes_df, child="Queue_Overflow", parents=["Delay_Class"])

parents_reliability = ["Incorrectly_Received", "Delay_Exceeded", "Queue_Overflow"]
reliability_cpt = cpt_probs(classes_df, child="Reliable", parents=parents_reliability)
reliability_cpt.rename(columns = {True:'True'}, inplace = True)
reliability_cpt.rename(columns = {False:'False'}, inplace = True)

In [4]:
# Save as CSV
data_type = "Downlink"
sinr_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}_v2/sinr_cpt.csv".format(data_type))
ber_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}_v2/ber_cpt.csv".format(data_type))
delay_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}_v2/delay_cpt.csv".format(data_type))
incorrect_rcvd_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}_v2/incorrect_rcvd_cpt.csv".format(data_type))
delay_exceeded_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}_v2/delay_exceeded_cpt.csv".format(data_type))
queue_overflow_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}_v2/queue_overflow_cpt.csv".format(data_type))
reliability_cpt.to_csv("/home/research-student/omnet-fanet/cpt/{}_v2/reliability_cpt.csv".format(data_type))

# Save as pickle
sinr_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}_v2/sinr_cpt.pkl".format(data_type))
ber_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}_v2/ber_cpt.pkl".format(data_type))
delay_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}_v2/delay_cpt.pkl".format(data_type))
incorrect_rcvd_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}_v2/incorrect_rcvd_cpt.pkl".format(data_type))
delay_exceeded_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}_v2/delay_exceeded_cpt.pkl".format(data_type))
queue_overflow_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}_v2/queue_overflow_cpt.pkl".format(data_type))
reliability_cpt.to_pickle("/home/research-student/omnet-fanet/cpt/{}_v2/reliability_cpt.pkl".format(data_type))